In [1]:
import random
import numpy as np
import seaborn as sn
import tensorflow as tf
import ipywidgets as widgets
import matplotlib.pyplot as plt

from ipywidgets import Output
from tensorflow.keras.datasets import mnist
from IPython.display import display
from mpl_toolkits.mplot3d import axes3d
from matplotlib.colors import ListedColormap
from sklearn import decomposition, preprocessing
from ipywidgets import interact, interactive, fixed, interact_manual

%matplotlib widget

In [2]:
# load (downloaded if needed) the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

Model Metrics:

In [3]:
# Import trained model

model_test = X_test / 255.0

model = tf.keras.models.load_model('mnist_model.h5')
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(model_test)
metrics = model.evaluate(model_test, y_test, steps=10000, verbose = 0)
print('Loss: ' + str(round(metrics[0], 5)))
print('Accuracy: ' + str(round(metrics[1], 4)))
#print(np.argmax(predictions[0]))
def classify(index):
    return np.argmax(predictions[index])

Loss: 0.07102
Accuracy: 0.9809


In [4]:

def create_eval_plot(): #Populates output widget with the plot display information
    fig1 = plt.figure("Random Set of Test Images")
    plt.clf()
    #Show 4 images from the test set in a 2x2 grid
    rand_temp = random.randint(0,10000)
    fig1.add_subplot(221).title.set_text('Model guess: ' + str(classify(rand_temp)) + '\n Correct label: ' + str(y_test[rand_temp]))
    plt.imshow(X_test[rand_temp], cmap=plt.get_cmap('gray'))
    rand_temp = random.randint(0,10000)
    fig1.add_subplot(222).title.set_text('Model guess: ' + str(classify(rand_temp)) + '\n Correct label: ' + str(y_test[rand_temp]))
    plt.imshow(X_test[rand_temp], cmap=plt.get_cmap('gray'))
    rand_temp = random.randint(0,10000)
    fig1.add_subplot(223).title.set_text('Model guess: ' + str(classify(rand_temp)) + '\n Correct label: ' + str(y_test[rand_temp]))
    plt.imshow(X_test[rand_temp], cmap=plt.get_cmap('gray'))
    rand_temp = random.randint(0,10000)
    fig1.add_subplot(224).title.set_text('Model guess: ' + str(classify(rand_temp)) + '\n Correct label: ' + str(y_test[rand_temp]))
    plt.imshow(X_test[rand_temp], cmap=plt.get_cmap('gray'))
    
    fig1.subplots_adjust(hspace=0.5)
    plt.show()
eval_out = widgets.Output()
with eval_out:
    out = widgets.interact_manual(create_eval_plot)
    out.widget.children[0].description = 'New Random Set'

In [5]:
counts = np.bincount(y_train) #Counts occurences of each digit
bar_out = widgets.Output()
with bar_out: #Populates output widget with the plot display information
    fig4 = plt.figure('Number Frequency in Training Set')
    ax = plt.subplot()
    ax.bar(range(10), counts, width=0.8, align='center')
    ax.set(xticks=range(10), xlim=[-1, 10])
    fig4.add_subplot(ax)

    plt.show()

In [6]:
#Turn each training image into a 1x784 array for processing
data = np.reshape(X_train, (60000, 784))

#Reduce the 784 dimentions into 3 principle components, making it a 60,000x3 array
standardized_data = preprocessing.StandardScaler().fit_transform(data)
pca=decomposition.PCA()
pca.n_components = 3
pca_data = pca.fit_transform(standardized_data)

#Generate color palette
cmap = ListedColormap(sn.color_palette("husl", 256).as_hex())

#Choose 1000 samples to display, using all 60,000 was too slow. 
#index is the starting index.
window = 0

window_slider = widgets.IntSlider(value = 1000, min = 100, max = 60000, step = 100)
index_slider = widgets.IntSlider(value = 0, min = 0, max = 59000, step = 100)

def update_index_max(*args):
    index_slider.max = 60000 - window_slider.value
window_slider.observe(update_index_max, 'value')

scatter_out = widgets.Output()

def draw_scatter(window, start, pca_data):

    pca_data = pca_data[start:start+window]
    labels = y_train[start:start+window]

    #Swap array dimensions so the principle components can be fed to the plot
    pca_data = pca_data.T 
    fig = plt.figure('Principle Component Analysis')
    plt.clf()
    ax = axes3d.Axes3D(fig, auto_add_to_figure=False)

    fig.add_axes(ax)
    #Sets each principle component as an axis, assignes color based on the label,
    #and uses the palette that was previously generated
    sc = ax.scatter(pca_data[0], pca_data[1], pca_data[2], c=labels, cmap = cmap)

    plt.legend(*sc.legend_elements(),  loc=1) #Generates legend

    plt.show()
with scatter_out:
    interact(draw_scatter, window=window_slider, start=index_slider, pca_data=fixed(pca_data))

In [7]:
tab = widgets.Tab()
tab.children = [scatter_out, bar_out, eval_out]
tab.set_title(0, 'PCA')
tab.set_title(1, 'Bar Chart')
tab.set_title(2, 'Evaluation')
display(tab)